## **Import necessary libraries**

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import pandas as pd
import numpy as np
import librosa
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os

## **Load data**

In [3]:
path='C:/Users/ASUS/Downloads/FSDKaggle2018/'

# Read metadata file
metadata_file = path + 'FSDKaggle2018.meta/train_post_competition.csv'
# print(metadata_file)
# contents = os.listdir('/content/drive/MyDrive/FSDKaggle2018')

# for item in contents:
#     print(item)
train = pd.read_csv(metadata_file)

# Take relevant columns (features, labels)
train = train[['fname', 'freesound_id', 'label']]
print(train)

# Audio training data path
data_path = path + 'FSDKaggle2018l.audio_train/'


             fname  freesound_id         label
0     00044347.wav         28739        Hi-hat
1     001ca53d.wav        358827     Saxophone
2     002d256b.wav         10897       Trumpet
3     0033e230.wav        325017  Glockenspiel
4     00353774.wav        195688         Cello
...            ...           ...           ...
9468  ffec59fb.wav        343090     Fireworks
9469  fff37590.wav         33136        Hi-hat
9470  fff44ac6.wav        133674      Laughter
9471  fff6a13d.wav         14640         Chime
9472  fff81f55.wav         19117         Cough

[9473 rows x 3 columns]


In [4]:
IMG_SIZE = (128, 128)
TRAIN_PATH = path + '/FSDKaggle2018.audio_train/'
TEST_PATH = path + '/FSDKaggle2018.audio_test/'
batch_size = 32

## **Load datasets**

#### Create custom dataset class

In [5]:
class SoundDataset(Dataset):
    def __init__(self, dataframe, path, test=False):
        super(SoundDataset, self).__init__()
        self.dataframe = dataframe
        self.path = path
        self.test = test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        file_path = self.dataframe.fname.values[idx]
        label = self.dataframe.label.values[idx]
        path = (TEST_PATH if self.test else TRAIN_PATH) + file_path
        signal, _ = librosa.load(path)
        signal = librosa.feature.melspectrogram(y=signal)
        signal = librosa.power_to_db(signal, ref=np.max)

        try:
            resized = cv2.resize(signal, (IMG_SIZE[1], IMG_SIZE[0]))
        except Exception as e:
            print(path)
            print(str(e))
            resized = np.zeros(shape=(IMG_SIZE[1], IMG_SIZE[0]))

        X = np.stack([resized] * 3)  # Дублирование каналов
        X = torch.tensor(X, dtype=torch.float32)

        if not self.test:
            y = label_encoder[label]
            return X, y
        else:
            return X

#### Divide data into datasets

In [8]:
x_train, x_validation, y_train, y_validation = train_test_split(train, train, test_size=0.2, shuffle=True, random_state=5)
train_dataset = SoundDataset(x_train, TRAIN_PATH)
val_dataset = SoundDataset(x_validation, TRAIN_PATH)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [9]:
print(x_train.head())
print(y_train.head())

             fname  freesound_id     label
4882  83953b78.wav        216298   Shatter
4416  77d93f2b.wav         30045  Laughter
4481  79b29f52.wav        176537     Cough
6589  b41d03ad.wav        207124      Bark
1033  1c6b07b8.wav        372776     Cello
             fname  freesound_id     label
4882  83953b78.wav        216298   Shatter
4416  77d93f2b.wav         30045  Laughter
4481  79b29f52.wav        176537     Cough
6589  b41d03ad.wav        207124      Bark
1033  1c6b07b8.wav        372776     Cello


In [10]:
labels = np.unique(train.label.values)
label_encoder = {label:i for i, label in enumerate(labels)}

## **Build model**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = torchvision.models.efficientnet_b0(pretrained=True)
model.classifier[1] = torch.nn.Linear(1280, 41)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(optimizer)
cost = torch.nn.CrossEntropyLoss()
print(cost)

## **Train model**

#### Training loop

In [11]:
def train(epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = cost(pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X.size(0)
            train_correct += torch.sum(pred.argmax(1) == y).item()

        train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = train_correct / len(train_loader.dataset)

        model.eval()
        val_loss = 0
        val_correct = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                loss = cost(pred, y)
                val_loss += loss.item() * X.size(0)
                val_correct += torch.sum(pred.argmax(1) == y).item()

        val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)

        print("Epoch {}, Train Loss: {:.4f}, Train Accuracy: {:.4f}, Val Loss: {:.4f}, Val Accuracy: {:.4f}".format(epoch+1, train_loss, train_accuracy, val_loss, val_accuracy))

#### Train for a number of epochs

In [18]:
num_epochs = 5
train(num_epochs)

Epoch 1, Train Loss: 0.1235, Train Accuracy: 0.9616, Val Loss: 1.0704, Val Accuracy: 0.7931
Epoch 2, Train Loss: 0.1323, Train Accuracy: 0.9588, Val Loss: 1.1565, Val Accuracy: 0.7831
Epoch 3, Train Loss: 0.1323, Train Accuracy: 0.9588, Val Loss: 1.1588, Val Accuracy: 0.7842
Epoch 4, Train Loss: 0.1106, Train Accuracy: 0.9677, Val Loss: 1.0875, Val Accuracy: 0.7984
Epoch 5, Train Loss: 0.0846, Train Accuracy: 0.9718, Val Loss: 1.1974, Val Accuracy: 0.7794


## **Evaluate model**

In [ ]:
test_dataset = SoundDataset(test, TEST_PATH, test=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
predictions = torch.tensor([])

In [ ]:
model.eval()
with torch.no_grad():
    for X in test_loader:
        X = X.to(device)
        y_hat = model(X)
        predictions = torch.cat([predictions, y_hat.cpu()])

predictions = F.softmax(predictions, dim=1).detach().numpy()

submission_top1 = test.copy()

## **Save model**

In [14]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

#### Save

In [17]:
# Save the entire model, including the state_dict, optimizer state, and other information
torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': cost,
}, 'models/modelV1.1.pth')


#### Load

In [15]:
# Load the model
checkpoint = torch.load('models/modelV1.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
model